In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#load in csv with species abundances
abundances = pd.read_csv("abundances.csv")
abundances.head()

In [ ]:
#assign the number of rows and columns in the dataframe to variables 
row,col = abundances.shape

In [ ]:
#create a dictionary of the species with the locality and abundance data 
sppdict = {}
for i in range(col):
    #if i%2==1, then the column is a data column 
    #if i%2==0, then it is a species column 
    if i%2 ==1:
        #look up the species at i-1 and the abundance at i 
        #strip the species of any spaces so that they can all be combined later
        species = abundances.iloc[:,i-1].str.strip()
        counts = abundances.iloc[:,i]
        locality = list(abundances)[i-1]
        
        for j in range(row):
            #create a dictionary in which each entry is a species name 
            ind_species = species[j]
            ind_count = counts[j]
            get = sppdict.get(ind_species, None)
            
            if get== None:
                #if there is no entry (which there shouldn't be because it's a blank dictionary), add the locality name and the abundance count for that species 
                sppdict[ind_species] = [(locality, ind_count)]
            else:
                #if there is an entry, add the locality and individual count info
                sppdict[ind_species] = sppdict[ind_species] + [(locality, ind_count)]

In [ ]:
#create a blank list
sample = []
for i in range (col):
    #make a list of the localities (which are all the odd columns)
    if i%2 ==0:
        sample.append(list(abundances)[i])
#create a new, empty dataframe in which the column names are locality names 
df1 = pd.DataFrame(columns = sample)
df1.head()

In [ ]:
#fill in counts based on locality and species
#sppdict.keys() gives you all the species names 
for i in sppdict.keys():
    #this gives you a list of the values (locality, abundance)
    value = sppdict[i]
    
    #assign locality and count to variables 
    for j,k in value:
        locality = j
        count = k
        
        #fill in the blank dataframe with the count based on the index of species name, locality 
        df1.at[i, locality] = k   

In [ ]:
#Fill NaNs with 0
df1 = df1.fillna(0)

In [ ]:
#sort the dataframe by species name 
df1 = df1.sort_index()
df1 = df1.reindex(sorted(df1.columns), axis = 1)

In [ ]:
#export the merged dataframe and use that csv to fix any typos
df1.to_csv('abundances_merged.csv')
#Go check species names and fix any issues - save as abundances_new.csv

In [ ]:
#read in the new dataframe with the fixed typos
df2 = pd.read_csv("abundances_edited.csv")
df2 = df2.set_index('Species')
#combine the species now that they were fixed - grouby prevents the data from being lost but instead adds the values for each column together as they merge
df2 = df2.groupby(df2.index).sum()

In [ ]:
#transpose so that columns are equal to species (attributes) and rows are localities (samples)
df2 = df2.transpose()

In [ ]:
#export as the cleaned and merged dataframe 
df2.to_csv('abundances_cleaned.csv')
df2.head()

In [ ]:
#load in csv with list of all bivalves in dataset
bivalves = pd.read_csv("Bivalvia_noSD.csv")

In [ ]:
#convert csv to list 
bivalvia = list(bivalves['Bivalves'])

In [ ]:
#create a new dataframe in which all bivalve abundances are divided by 2
df3 = df2.copy()
for i in bivalvia:
    df3[i] = df2[i].divide(2).apply(np.ceil)  

In [ ]:
#create a new dataframe in which all abundances are relative proportions (species abundance/total individual count)
df4 = df3.divide(df2.sum(axis = 1), axis = 0)


In [ ]:
#export as proportion abundances
df4.to_csv("Proportional_Abundances.csv")

In [ ]:
#export as raw abundances
df3.to_csv("Final_Raw_Abundances.csv")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install scikit-bio
from sklearn import manifold


In [ ]:
#read in abundance csv
original = pd.read_csv('Proportional_Abundances.csv')
original.head()
#set index as localities
original = original.set_index('Localities')
#convert to a numpy array
a_original = np.array(original)
#set the localities as the indices
ids = original.index

In [ ]:
#load in numpy array of bray curtis distances
bc = np.load('bcdistances.npy')
#check shape of the array
type(bc)

In [ ]:
#convert the numpy array to a dataframe to make further steps easier
bc_df = pd.DataFrame(data = bc, index = ids, columns = ids)

In [ ]:
#import necessary packages for NMDS
import skbio.diversity
from skbio.diversity import beta_diversity
from skbio import DistanceMatrix
bc = beta_diversity("braycurtis", a_original, ids = ids)

In [ ]:
#calculate the nmds using manifold.MDS, specify 'false' for metric because it is non-metric
#specify 'precomputed' for dissimilarity because already computed as bray curtis
nmds = manifold.MDS(n_components = 2, metric = "false", max_iter = 3000, eps = 1e-12, 
                     dissimilarity = 'precomputed', n_jobs = 1)
npos = nmds.fit_transform(bc_df)
#specify the columns
cols = ['x','y']
#make the coordinates into a dataframe 
coords = pd.DataFrame(data = npos, columns = cols)
coords['Localities'] = ids

In [ ]:
#read in the previously created lookup table 
lookup = pd.read_csv('lookup.csv')

In [ ]:
#Copy the x and y coordinates into their own variables 
x = coords[['Localities', 'x']].copy()
y = coords[['Localities', 'y']].copy()

In [ ]:
#make the coordinates into dictionaries
x_dict = x.set_index('Localities')['x'].to_dict()
y_dict = y.set_index('Localities')['y'].to_dict()

In [ ]:
#map the dictionaries to the lookup table
lookup['x'] = lookup['Field #'].map(x_dict.get)
lookup['y'] = lookup['Field #'].map(y_dict.get)

In [ ]:
#set index as the Field # in the lookup table and sort by the Formation and Locality
lookup = lookup.set_index('Field #')
lookup = lookup.sort_values(by = ['Formation', 'Locality'])

In [ ]:
#make independent dataframes for each geological formation
careaga = lookup[lookup["Formation"] == 'Careaga']
pico = lookup[lookup["Formation"] == 'Pico']
sandiego = lookup[lookup ["Formation"] == 'San Diego']

In [ ]:
#import seaborn for data visualization
import seaborn as sns

In [ ]:
#Plot in reduced ordination space and color code points by Locality 
p1 = sns.scatterplot(x = 'x', y = 'y', data = lookup, hue = 'Locality',
                style = "Formation", s = 50, palette = "magma")

plt.legend(bbox_to_anchor = (1.05,1), loc = 2, borderaxespad = 0)

for line in range(0, lookup.shape[0]):
    p1.text(lookup.x[line]+0.05, lookup.y[line], lookup.index[line], horizontalalignment='left', 
            size = 'small', color = 'black')
    
plt.savefig('NMDS.png', bbox_inches = 'tight', dpi = 100)


In [ ]:
#Plot only Careaga Sandstone samples in reduced ordination space

p3 = sns.scatterplot(x = 'x', y = 'y', style = "Locality", data = careaga, s = 50, palette = 'viridis')
plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad = 0)

for line in range(0, careaga.shape[0]):
    p3.text(careaga.x[line]+0.05, careaga.y[line], careaga.index[line], horizontalalignment='left', 
            size = 'small', color = 'black', weight = 'semibold')

In [ ]:
#Plot only Pico Formation samples in reduced ordination space
p8 = sns.scatterplot(x = 'x', y = 'y', data = pico, style = 'Locality', s = 50, palette = 'viridis')
plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad = 0)

for line in range(0, pico.shape[0]):
    p8.text(pico.x[line]+0.05, pico.y[line], pico.index[line], horizontalalignment='left', 
            size = 'small', color = 'black', weight = 'semibold')

In [ ]:
#calculate the kmeans on the original matrix
import scipy
from skbio.diversity import beta_diversity
from skbio import DistanceMatrix
bc = beta_diversity("braycurtis", a_original, ids = ids)
bc1 = DistanceMatrix.redundant_form(bc)

In [ ]:
kmeans = KMeans(n_clusters = 3).fit(bc1)
ykmeans = kmeans.fit_predict(bc1)
c = kmeans.cluster_centers_

In [ ]:
#create arrays for each cluster
bc1 = bc_df.copy()
bc1['Cluster'] = list(ykmeans)
cl0 = bc1[bc1["Cluster"] == 0]
cl1 = bc1[bc1["Cluster"] == 1]
cl2 = bc1[bc1["Cluster"] == 2]
cl0 = cl0.drop("Cluster", axis = 1)
cl1 = cl1.drop("Cluster", axis = 1)
cl2 = cl2.drop("Cluster", axis = 1)
cl0 = np.array(cl0)
cl1 = np.array(cl1)
cl2 = np.array(cl2)

In [ ]:
#calculate mean and std of clusters from their relative centers
cl_0 = np.abs(cl0 - c[0])
print(cl_0.mean(), cl_0.std())
cl_1 = np.abs(cl1 - c[1])
print(cl_1.mean(), cl_1.std())
cl_2 = np.abs(cl1 - c[2])
print(cl_2.mean(), cl_2.std())

In [ ]:
from scipy import stats

In [ ]:
#calculate the kmeans on the coordinates to visualize in 2D space
kmeans= KMeans(n_clusters = 3).fit(npos)
ykmeans = kmeans.fit_predict(npos)
c = kmeans.cluster_centers_

In [ ]:
#create a scatter plot of the coordinates using the kmeans analysis to see how they cluster
plt.scatter(npos[:,0], npos[:,1], c = ykmeans, s = 50, cmap = "viridis")
plt.scatter(c[:,0], c[:,1], marker = 'X', c = "R")
plt.savefig('kmeans.png', bbox_inches = 'tight', dpi = 100)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
clustering = AgglomerativeClustering(affinity='precomputed', compute_full_tree='auto',
                        connectivity=None,
                        linkage='single', memory=None, n_clusters=2,
                        pooling_func='deprecated').fit(bc1)
clustering
clustering.labels_

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt


bc_condensed = pdist(a_original, 'braycurtis')
Z = hierarchy.linkage(bc_condensed, 'single')
# Plot with Custom leaves
dn=hierarchy.dendrogram(Z, leaf_rotation=0, leaf_font_size=5, labels=original.index, orientation = 'right')
plt.gcf()
plt.savefig('dendrogram.png', dpi = 150)